# How click can be triggered based on callback_context or ctx

In [79]:
from dash import Dash, html, dcc, Input, Output, ctx, callback

app = Dash(__name__)

app.layout = html.Div([
    html.Button('Button 1', id='button_1'),
    html.Button('Button 2', id='button_2'),
    html.Button('Button 3', id='button_3'),
    html.Button('Button 4', id='button_4'),
    html.Div(id='container-ctx-example')
])


@callback(Output('container-ctx-example','children'),
              Input('button_1', 'n_clicks'),
              Input('button_2', 'n_clicks'),
              Input('button_3', 'n_clicks'),
              Input('button_4', 'n_clicks'),
         )
def display(btn1, btn2, btn3,btn4):
    button_clicked = ctx.triggered_id
    button_clicked_triggered = ctx.triggered_prop_ids
    print(button_clicked)
    return html.Div([
        dcc.Markdown(
            f'''You last clicked button with ID {button_clicked}
            You last clicked button with ID {button_clicked_triggered}
            You last clicked button with ID {ctx.triggered}
            You last clicked button with ID {ctx.inputs}
            ''' if button_clicked else '''You haven't clicked any button yet''')
    ])

if __name__ == '__main__':
    app.run(debug=True)


None
button_4


In [2]:
from dash import Dash, dcc, Output, Input, html, State
import dash
import dash_bootstrap_components as dbc
import plotly.express as px
import dash_daq as daq
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('/Users/mg_0328/Documents/ContentMapping/untitled folder/PLOTLY DASH/social_capital.csv')

get_attribute = {

    "edit": True,
    "color": {
        "Page Background Color": "paper_bgcolor",
        "Plot Background Color": "plot_bgcolor",
        "Font Color": "font_color",
        "Title Font Color": "title_font_color"

    }

}

graph_color = [
    {
        "color": {

        }
    }
]

app = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])
dummy = dcc.Markdown(children="dummy")
mytitle = dcc.Markdown(children="Selectable Bargraph")
mygraph = dcc.Graph(
    id="graph_1",
    figure={},
    config={"editable": True,
            # "editSelection": True,
            'modeBarButtonsToAdd': ['drawline',
                                    'drawopenpath',
                                    'drawclosedpath',
                                    'drawcircle',
                                    'drawrect',
                                    'eraseshape',
                                    # "resetViewMapbox"

                                    ]
            },

)
bg_color_checkbox = dcc.Checklist(
    list(get_attribute["color"].keys()),
    ['Page Background Color', "Plot Background Color"]
)
color_picker = daq.ColorPicker(
    id='my-color-picker-1',
    label='Color Picker',
    value=dict(hex='#119DFF')
)

toggle_block = html.Div(
    [
        daq.BooleanSwitch(
            label='Grid Lines',
            id="bool-switch-grid",
            on=True,
            color="#9B51E0",
        ),
        daq.BooleanSwitch(
            label='H-Grid Line',
            id="bool-switch-grid-h",
            on=True,
            color="#9B51E0",
        ),
        daq.BooleanSwitch(
            label='V-Grid Line',
            id="bool-switch-grid-v",
            on=True,
            color="#9B51E0",
        ),
        daq.BooleanSwitch(
            label='Legend',
            id="bool-switch-legend",
            on=True,
            color="#9B51E0",
        )

    ], style={"padding": "15px"}
)

graph_block = html.Div([mygraph,
                        toggle_block

                        ], id="graph-block", style={
    "display": "flex",
    "flex-direction": "row",
    "flex-wrap": "nowrap",
    "justify-content": "space-evenly",
    "align-content": "center"
})

drop_down = dcc.Dropdown(options=df.describe().columns[1:], value=df.describe().columns[1], clearable=False)
app.layout = dbc.Container([
    mytitle,
    graph_block,
    drop_down,
    bg_color_checkbox,
    color_picker,
])


def update_dictionary_for_same_values(key, values, color):
    mappings = {}
    for i in values:
        mappings[key[i]] = color

    return mappings


@app.callback(
    Output(mygraph, component_property="figure"),
    Input(drop_down, component_property="value"),
    State(color_picker, component_property="value"),
    State(bg_color_checkbox, component_property="value")
)
def update_drop_down(user_input, color_name, checkbox_color):
    #     print(color_name, "printing color")
    d = df[df["STATE"].isin(['AL', "AK", "ME"])]
    fig = px.bar(data_frame=d, x='YEAR', y=user_input, color="STATE")
    #     print(checkbox_color)

    update_dict = update_dictionary_for_same_values(get_attribute['color'], checkbox_color, color_name["hex"])
    fig.update_layout(update_dict)
    #     fig.update_layout({
    #         "paper_bgcolor":color_name["hex"],
    #         "plot_bgcolor":color_name["hex"]
    #                           })
    return fig


@app.callback(
    Output(mygraph, component_property="figure", allow_duplicate=True),
    Input(color_picker, component_property="value"),
    State(bg_color_checkbox, component_property="value"),
    State(mygraph, component_property="figure"),
    prevent_initial_call=True
)
def update_graph(color_name, checkbox_color, fig):
    #     print(color_name)
    fig = go.Figure(fig)
    update_dict = update_dictionary_for_same_values(get_attribute['color'], checkbox_color, color_name["hex"])
    fig.update_layout(update_dict)
    return fig


@app.callback(
    Output(mygraph, component_property="figure", allow_duplicate=True),
    Input(mygraph, component_property="figure"),
    Input("bool-switch-grid", component_property="on"),
    Input("bool-switch-grid-h", component_property="on"),
    Input("bool-switch-grid-v", component_property="on"),
    prevent_initial_call=True
)
def update_axis(fig, on, hon, von):
    #     print(on)
    fig = go.Figure(fig)
    fig.update_layout(xaxis=dict(showgrid=on & von),
                      yaxis=dict(showgrid=on & hon))

    return fig


# @app.callback(
#     Output(mygraph, component_property="figure", allow_duplicate=True),
#     Input(mygraph, component_property="figure"),
#     Input("bool-switch-legend", component_property="on"),
#     prevent_initial_call=True
# )
# def toggle_legend(fig, on):
#     print(on)
#     fig = go.Figure(fig)
#     fig.update_layout(showlegend=on)

#     return fig


@app.clientside_callback("""

    function(fig, on):
    print(on)
    fig = go.Figure(fig)
    fig.update_layout(showlegend=on)

    return fig

""",
    Output(mygraph, component_property="figure", allow_duplicate=True),
    [ Input(mygraph, component_property="figure"),
    Input("bool-switch-legend", component_property="on")],
    prevent_initial_call=True)



if __name__ == "__main__":
    app.run_server(debug=True)




SyntaxError: invalid syntax (2811791013.py, line 205)

In [85]:
import pandas as pd
df=pd.read_clipboard()
df

Program  Year        Period    State  ANSI        Affected by  \
0     SURVEY  2019  JAN THRU MAR  Alabama     1            Disease   
1     SURVEY  2019  JAN THRU MAR  Alabama     1              Other   
2     SURVEY  2019  JAN THRU MAR  Alabama     1         Pesticides   
3     SURVEY  2019  JAN THRU MAR  Alabama     1  Pests_excl_Varroa   
4     SURVEY  2019  JAN THRU MAR  Alabama     1            Unknown   
...      ...   ...           ...      ...   ...                ...   
4585  SURVEY  2015  OCT THRU DEC  Wyoming    56              Other   
4586  SURVEY  2015  OCT THRU DEC  Wyoming    56         Pesticides   
4587  SURVEY  2015  OCT THRU DEC  Wyoming    56  Pests_excl_Varroa   
4588  SURVEY  2015  OCT THRU DEC  Wyoming    56            Unknown   
4589  SURVEY  2015  OCT THRU DEC  Wyoming    56       Varroa_mites   

      Pct of Colonies Impacted state_code  
0                          1.8         AL  
1                          3.1         AL  
2                          0.3         AL  
3                         22.7         AL  
4                          9.0         AL  
...                        ...        ...  
4585                      11.3         WY  
4586                      10.5         WY  
4587                       3.3         WY  
4588                       3.9         WY  
4589                      45.0         WY  

[4590 rows x 8 columns]

In [3]:
# Solution to bar challenge.
# Changed code lines 64 to 71

import pandas as pd
import plotly.express as px  # (version 4.7.0)

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

# ------------------------------------------------------------------------------
# Import and clean data (importing csv into pandas)
# df = pd.read_csv("intro_bees.csv")

df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
print(df[:5])

# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2015", "value": 2015},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={})

])


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["Year"] == option_slctd]
    dff = dff[dff["Affected by"] == "Varroa_mites"]

    fig = px.bar(
        data_frame=dff,
        x='State',
        y='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )

    return container, fig


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=True)

/var/folders/6n/znxyq30x5svfn7c24p94vcfctqyqqt/T/ipykernel_33643/889519818.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/6n/znxyq30x5svfn7c24p94vcfctqyqqt/T/ipykernel_33643/889519818.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


NameError: name 'df' is not defined

In [89]:
import dash
import plotly.express as px
import pandas as pd

# Data Exploration with Pandas (python)
# -----------------------------------------------------------------

df = pd.read_csv("vgsales.csv") # data by GregorySmith from kaggle

print(df[:5])
print(df.iloc[:5, [2,3,5,10]])
print(df.Genre.nunique())
print(df.Genre.unique())
print(sorted(df.Year.unique()))

# Data Visualization with Plotly (Python)
# -----------------------------------------------------------------

fig_pie = px.pie(data_frame=df, names='Genre', values='Japan Sales')
fig_pie = px.pie(data_frame=df, names='Genre', values='North American Sales')
fig_pie.show()

fig_bar = px.bar(data_frame=df, x='Genre', y='Japan Sales')
fig_bar.show()

fig_hist = px.histogram(data_frame=df, x='Year', y='Japan Sales')
fig_hist.show()

# Interactive Graphs with Dash (Python, R, Julia)
# -----------------------------------------------------------------

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input

app = dash.Dash(__name__)

app.layout=html.Div([
    html.H1("Graph Analysis with Charming Data"),
    dcc.Dropdown(id='genre-choice',
                 options=[{'label':x, 'value':x}
                          for x in sorted(df.Genre.unique())],
                 value='Action'
                 ),
    dcc.Graph(id='my-graph',
              figure={}),
])
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='genre-choice', component_property='value')
)
def interactive_graphs(value_genre):
    print(value_genre)
    dff = df[df.Genre==value_genre]
    fig = px.bar(data_frame=dff, x='Year', y='Japan Sales')
    return fig


if __name__=='__main__':
    app.run_server()

   Rank                  Video Game Platform  Year    Genre  \
0   222                     FIFA 17      PS4  2016   Sports   
1   272  Uncharted 4: A Thief's End      PS4  2016  Shooter   
2   352   Tom Clancy's The Division      PS4  2016  Shooter   
3   772             Far Cry: Primal      PS4  2016   Action   
4   847   Tom Clancy's The Division     XOne  2016  Shooter   

                     Publisher  North American Sales  EU Sales  Japan Sales  \
0              Electronic Arts                  0.28      3.75         0.06   
1  Sony Computer Entertainment                  1.30      2.07         0.18   
2                      Ubisoft                  1.28      1.61         0.15   
3                      Ubisoft                  0.59      1.16         0.06   
4                      Ubisoft                  1.20      0.62         0.00   

   Other Sales  World Sales  
0         0.69         4.77  
1         0.65         4.20  
2         0.57         3.61  
3         0.33         2.1

Action


In [1]:
import plotly.express as px
import plotly.graph_objects as go
import json
import joblib
import plotly

In [65]:
g=open("/Users/mg_0328/Downloads/PLOTS/")
data = json.load(g)
data

{'data': [{'alignmentgroup': 'True',
   'hovertemplate': 'ATTEND=%{x}<br>BMI=%{y}<extra></extra>',
   'legendgroup': '',
   'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
   'name': '',
   'offsetgroup': '',
   'orientation': 'v',
   'showlegend': False,
   'textposition': 'auto',
   'x': [3,
    1,
    1,
    3,
    1,
    1,
    1,
    1,
    1,
    3,
    1,
    1,
    1,
    1,
    1,
    2,
    1,
    1,
    1,
    2,
    1,
    1,
    1,
    3,
    1,
    1,
    3,
    2,
    3,
    1,
    1,
    1,
    3,
    1,
    3,
    1,
    1,
    1,
    1,
    3,
    1,
    3,
    3,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    3,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    3,
    1,
    1,
    1,
    1,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    2,
    1,
    1,
    3,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    2,
    1,
    1,
    1,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
  

In [133]:
df.groupby("ATTEND").mean().reset_index()

ATTEND        BMI
0  MAYBE  28.948191
1     NO  28.904590
2    YES  28.938707

In [2]:
import plotly.express as px

long_df = px.data.medals_long()
long_df

nation   medal  count
0  South Korea    gold     24
1        China    gold     10
2       Canada    gold      9
3  South Korea  silver     13
4        China  silver     15
5       Canada  silver     12
6  South Korea  bronze     11
7        China  bronze      8
8       Canada  bronze     12

In [8]:
fig = px.bar(long_df, x="nation", y="count",color="medal",)
fig.show(config={"scrollZoom":True})


In [34]:
# fig_dict=fig.to_dict()
type(fig_dict)

dict

In [37]:
json.dumps(fig_dict)

TypeError: Object of type ndarray is not JSON serializable

In [29]:
fig_dict["config"]={"scrollZoom":True}

In [40]:
from jsonify import convert

In [62]:
fig.to_json()

'{"data":[{"alignmentgroup":"True","hovertemplate":"medal=gold<br>nation=%{x}<br>count=%{y}<extra></extra>","legendgroup":"gold","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"gold","offsetgroup":"gold","orientation":"v","showlegend":true,"textposition":"auto","x":["South Korea","China","Canada"],"xaxis":"x","y":[24,10,9],"yaxis":"y","type":"bar"},{"alignmentgroup":"True","hovertemplate":"medal=silver<br>nation=%{x}<br>count=%{y}<extra></extra>","legendgroup":"silver","marker":{"color":"#EF553B","pattern":{"shape":""}},"name":"silver","offsetgroup":"silver","orientation":"v","showlegend":true,"textposition":"auto","x":["South Korea","China","Canada"],"xaxis":"x","y":[13,15,12],"yaxis":"y","type":"bar"},{"alignmentgroup":"True","hovertemplate":"medal=bronze<br>nation=%{x}<br>count=%{y}<extra></extra>","legendgroup":"bronze","marker":{"color":"#00cc96","pattern":{"shape":""}},"name":"bronze","offsetgroup":"bronze","orientation":"v","showlegend":true,"textposition":"auto","x"

In [73]:
# plotly.io.to_json(fig,"/Users/mg_0328/Downloads/PLOTS/bar_plot_13_09_part_2.json")
# plotly.io.write_json()
plotly.io.write_json(fig,'/Users/mg_0328/Downloads/PLOTS/bar_chart_13_09_part_2.json')
# (fig_dict)

In [75]:
json.dumps(fig_dict)

TypeError: Object of type ndarray is not JSON serializable

In [92]:
fig_dict.__dict__

AttributeError: 'dict' object has no attribute '__dict__'

In [91]:
joblib.dump(fig_dict,'/Users/mg_0328/Downloads/PLOTS/bar_chart_13_09_part_2.json')

['/Users/mg_0328/Downloads/PLOTS/bar_chart_13_09_part_2.json']

In [94]:
import json
import numpy as np
class json_serialize(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
j=json.dumps(fig_dict, cls=json_serialize) 
    
with open(r"/Users/mg_0328/Downloads/PLOTS/bar_plot_13_09_1.json","w") as json_file:
    json.dump(j,json_file,indent=1)

In [100]:
import orjson

In [123]:
def _orjson_converter(obj):
    """Custom serializer/converter, e.g. for numpy object arrays."""
    print(obj)
    if isinstance(obj, np.ndarray) and obj.dtype == np.object_:
        return obj.tolist()

In [144]:
plotly.io.write_json(fig,"../../../../Downloads/PLOTS/plotly-poc/src/json_plotly/bar_chart_color_bg.json")

In [149]:

fig.update_layout({"paper_bgcolor":'red'})

In [150]:
fig.update_layout({"plot_bgcolor":'blue'})

In [151]:
plotly.io.write_json(fig,"/Users/mg_0328/Downloads/PLOTS/plotly-poc/src/json_plotly/bar_chart_full_json.json")



In [177]:
plotly.io.write_json(fig_,"/Users/mg_0328/Downloads/PLOTS/plotly-poc/src/json_plotly/scatter_data_label.json")


In [174]:
fig.update_traces( textposition='top center')

ValueError: 
    Invalid value of type 'builtins.str' received for the 'textposition' property of bar
        Received value: 'top center'

    The 'textposition' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['inside', 'outside', 'auto', 'none']
      - A tuple, list, or one-dimensional numpy array of the above

In [191]:
fig.update_annotations({"plot_bgcolor":"green"})

In [182]:
import plotly.express as px

df = px.data.gapminder().query("year==2007 and continent=='Americas'")

fig_ = px.scatter(df, x="gdpPercap", y="lifeExp",text="lifeExp")

fig_.update_traces(textposition='top center',textfont_size=1)

fig_.update_layout(
    height=800,
    title_text='GDP and Life Expectancy (Americas, 2007)'
)

fig_.show(
#     config={"scrollZoom":True}
)

In [137]:
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=False)
fig

In [192]:
import pandas as pd
import seaborn as sns
import plotly.express as px

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/mg_0328/Downloads/Task/Regression/regression_BMI_44C_10mb.csv')[["ATTEND","BMI"]]
df.head()

ATTEND   BMI
0       3  24.7
1       1  33.3
2       1  27.5
3       3  20.6
4       1  25.1

In [207]:
grouped.loc[grouped['total_bill'] == grouped['total_bill'].min()]['day'][1]

'Fri'

In [128]:
fig=px.scatter(data_frame=df,x="ATTEND",y="BMI")
fig

NameError: name 'df' is not defined

In [206]:
df = sns.load_dataset('tips')
grouped = df.groupby('day', as_index=False).sum()
display(grouped)
# grouped
fig = px.bar(data_frame=grouped, x='day', y='total_bill')
fig.show()

/var/folders/6n/znxyq30x5svfn7c24p94vcfctqyqqt/T/ipykernel_61061/911618247.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



day  total_bill     tip  size
0  Thur     1096.33  171.83   152
1   Fri      325.88   51.96    40
2   Sat     1778.40  260.40   219
3   Sun     1627.16  247.39   216

In [204]:
annotation = {
    # x -> location for x
    'x': grouped["day"],
    # y -> location for y
    'y': grouped["tip"],
    'text': 'Bu güne dikkat etmeliyiz',  # text
    'showarrow': True,  # would you want to see arrow
    'arrowhead': 3,  # which type for arrowhead
    'font': {'size': 10, 'color': 'black'}  # font style
}

In [205]:
fig.add_annotation(annotation)
fig.show()

In [102]:
dict_=fig.to_dict()
dict_["layout"]["xaxis"]["showgrid"]=True
dict_["layout"]["xaxis"]["showgrid"]=True

In [108]:
fig.to_json('/Users/mg_0328/Downloads/PLOTS/scatter_plot.json')

'{"data":[{"hovertemplate":"ATTEND=%{x}<br>BMI=%{y}<extra></extra>","legendgroup":"","marker":{"color":"#636efa","symbol":"circle"},"mode":"markers","name":"","showlegend":false,"x":[3,1,1,3,1,1,1,1,1,3,1,1,1,1,1,2,1,1,1,2,1,1,1,3,1,1,3,2,3,1,1,1,3,1,3,1,1,1,1,3,1,3,3,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,3,1,1,1,1,2,1,1,1,1,1,1,1,1,2,1,1,3,1,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,1,2,1,1,3,1,1,1,1,3,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,3,1,1,3,3,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,2,1,1,1,1,1,1,1,1,1,4,1,1,1,1,1,2,1,1,1,1,1,1,1,2,1,1,1,3,1,1,3,1,3,1,1,1,1,1,3,1,1,1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,3,1,1,1,1,1,3,1,2,1,1,1,1,1,1,1,3,1,3,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,3,1,3,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,2,1,1,1,3,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,3,1,1,1,1,1,4,3,3,1,1,1,2

In [214]:
df.to_json("../../../../Downloads/PLOTS/plotly-poc/src/json_plotly/df.json")

In [224]:
fig_.to_dict()["layout"].keys()

dict_keys(['template', 'xaxis', 'yaxis', 'legend', 'margin', 'title', 'height', 'clickmode'])

In [221]:
fig_.update_layout({"clickmode":"select"}).

In [218]:
fig.update_layout({"clickmode":"select"})

In [217]:
fig.to_dict()

{'data': [{'alignmentgroup': 'True',
   'hovertemplate': 'day=%{x}<br>total_bill=%{y}<extra></extra>',
   'legendgroup': '',
   'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
   'name': '',
   'offsetgroup': '',
   'orientation': 'v',
   'showlegend': False,
   'textposition': 'auto',
   'x': array(['Thur', 'Fri', 'Sat', 'Sun'], dtype=object),
   'xaxis': 'x',
   'y': array([1096.33,  325.88, 1778.4 , 1627.16]),
   'yaxis': 'y',
   'type': 'bar'}],
 'layout': {'template': {'data': {'histogram2dcontour': [{'type': 'histogram2dcontour',
      'colorbar': {'outlinewidth': 0, 'ticks': ''},
      'colorscale': [[0.0, '#0d0887'],
       [0.1111111111111111, '#46039f'],
       [0.2222222222222222, '#7201a8'],
       [0.3333333333333333, '#9c179e'],
       [0.4444444444444444, '#bd3786'],
       [0.5555555555555556, '#d8576b'],
       [0.6666666666666666, '#ed7953'],
       [0.7777777777777778, '#fb9f3a'],
       [0.8888888888888888, '#fdca26'],
       [1.0, '#f0f921']]}],
    'chor

In [215]:
df

total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]

In [225]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter(df, x="sepal_length", y="sepal_width", color="species",
                 labels={
                     "sepal_length": "Sepal Length (cm)",
                     "sepal_width": "Sepal Width (cm)",
                     "species": "Species of Iris"
                 },
                title="Manually Specified Labels")
fig.show()

In [226]:
fig.to_dict()

{'data': [{'hovertemplate': 'Species of Iris=setosa<br>Sepal Length (cm)=%{x}<br>Sepal Width (cm)=%{y}<extra></extra>',
   'legendgroup': 'setosa',
   'marker': {'color': '#636efa', 'symbol': 'circle'},
   'mode': 'markers',
   'name': 'setosa',
   'orientation': 'v',
   'showlegend': True,
   'x': array([5.1, 4.9, 4.7, 4.6, 5. , 5.4, 4.6, 5. , 4.4, 4.9, 5.4, 4.8, 4.8,
          4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1, 5.4, 5.1, 4.6, 5.1, 4.8, 5. ,
          5. , 5.2, 5.2, 4.7, 4.8, 5.4, 5.2, 5.5, 4.9, 5. , 5.5, 4.9, 4.4,
          5.1, 5. , 4.5, 4.4, 5. , 5.1, 4.8, 5.1, 4.6, 5.3, 5. ]),
   'xaxis': 'x',
   'y': array([3.5, 3. , 3.2, 3.1, 3.6, 3.9, 3.4, 3.4, 2.9, 3.1, 3.7, 3.4, 3. ,
          3. , 4. , 4.4, 3.9, 3.5, 3.8, 3.8, 3.4, 3.7, 3.6, 3.3, 3.4, 3. ,
          3.4, 3.5, 3.4, 3.2, 3.1, 3.4, 4.1, 4.2, 3.1, 3.2, 3.5, 3.1, 3. ,
          3.4, 3.5, 2.3, 3.2, 3.5, 3.8, 3. , 3.8, 3.2, 3.7, 3.3]),
   'yaxis': 'y',
   'type': 'scatter'},
  {'hovertemplate': 'Species of Iris=versicolor<br>Sepal 

In [233]:
import plotly.express as px

df = px.data.gapminder().query("continent == 'Oceania'")
display(df)
fig = px.line(df, x="year", y="gdpPercap", color="country")

# fig.update_layout(
#     title=dict(text="GDP-per-capita", font=dict(size=50), automargin=True, yref='paper')
# )

fig.show()


country continent  year  lifeExp       pop    gdpPercap iso_alpha  \
60      Australia   Oceania  1952   69.120   8691212  10039.59564       AUS   
61      Australia   Oceania  1957   70.330   9712569  10949.64959       AUS   
62      Australia   Oceania  1962   70.930  10794968  12217.22686       AUS   
63      Australia   Oceania  1967   71.100  11872264  14526.12465       AUS   
64      Australia   Oceania  1972   71.930  13177000  16788.62948       AUS   
65      Australia   Oceania  1977   73.490  14074100  18334.19751       AUS   
66      Australia   Oceania  1982   74.740  15184200  19477.00928       AUS   
67      Australia   Oceania  1987   76.320  16257249  21888.88903       AUS   
68      Australia   Oceania  1992   77.560  17481977  23424.76683       AUS   
69      Australia   Oceania  1997   78.830  18565243  26997.93657       AUS   
70      Australia   Oceania  2002   80.370  19546792  30687.75473       AUS   
71      Australia   Oceania  2007   81.235  20434176  34435.36744       AUS   
1092  New Zealand   Oceania  1952   69.390   1994794  10556.57566       NZL   
1093  New Zealand   Oceania  1957   70.260   2229407  12247.39532       NZL   
1094  New Zealand   Oceania  1962   71.240   2488550  13175.67800       NZL   
1095  New Zealand   Oceania  1967   71.520   2728150  14463.91893       NZL   
1096  New Zealand   Oceania  1972   71.890   2929100  16046.03728       NZL   
1097  New Zealand   Oceania  1977   72.220   3164900  16233.71770       NZL   
1098  New Zealand   Oceania  1982   73.840   3210650  17632.41040       NZL   
1099  New Zealand   Oceania  1987   74.320   3317166  19007.19129       NZL   
1100  New Zealand   Oceania  1992   76.330   3437674  18363.32494       NZL   
1101  New Zealand   Oceania  1997   77.550   3676187  21050.41377       NZL   
1102  New Zealand   Oceania  2002   79.110   3908037  23189.80135       NZL   
1103  New Zealand   Oceania  2007   80.204   4115771  25185.00911       NZL   

      iso_num  
60         36  
61         36  
62         36  
63         36  
64         36  
65         36  
66         36  
67         36  
68         36  
69         36  
70         36  
71         36  
1092      554  
1093      554  
1094      554  
1095      554  
1096      554  
1097      554  
1098      554  
1099      554  
1100      554  
1101      554  
1102      554  
1103      554

In [230]:
plotly.io.write_json(fig,'../../../../Downloads/PLOTS/plotly-poc/src/json_plotly/line_chart.json')